In [ ]:
import numpy as np
import scipy.linalg as la
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import geomstats as gs
import geomstats.geometry.spd_matrices as spd
import pickle
from scipy.signal import butter, lfilter
from geomstats.learning.frechet_mean import FrechetMean
from scipy.io import loadmat
import pandas as pd
from copy import deepcopy
from scipy.stats import wilcoxon
from copy import deepcopy


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.3f}".format

# Globally used things

In [ ]:
class RCSP:
    def __init__(self, metric, nchannels, clf):
        self.metric = metric
        self.nchannels = nchannels
        self.clf=clf()
        self.V=None
        self.n=None
    
    def estimateMeans(self, classSpecificCOV):
        if self.metric=="classic":
            class0_avg = sum(classSpecificCOV[0])/len(classSpecificCOV[0])
            class1_avg = sum(classSpecificCOV[1])/len(classSpecificCOV[1])
            return [class0_avg, class1_avg]
        elif self.metric=="AIRM":
            estimator = FrechetMean(spd.SPDMetricAffine(n=self.nchannels), max_iter=64)
        elif self.metric=="LEM":
            estimator = FrechetMean(spd.SPDMetricLogEuclidean(n=self.nchannels), max_iter=64)
        elif self.metric=="BW":
            estimator = FrechetMean(spd.SPDMetricBuresWasserstein(n=self.nchannels), max_iter=64) #doesn't work yet
        else:
            raise Exception("Not implemented metric")
            
        means = []
        
        for COV in classSpecificCOV:
            estimator.fit(COV)
            mean = estimator.estimate_
            means.append(mean)
        return means
    
    def separate_classes(self, X, Y):
        classSpecificCOV = []
        for i in range(2): 
            ind = np.where(Y==i)[0]
            classCOV = X[ind]
            classSpecificCOV.append(classCOV)
        return classSpecificCOV
    
    def CSP(self, means, n):
        _,V = la.eigh(means[0], means[0]+means[1])
        V = np.concatenate((V[:, :n], V[:, -n:]), axis=1)
        return V
    
    def applyCSP(self, trial, V):
        a = np.dot(np.dot(V.T, trial), V) 
        f = np.log(np.diagonal(a)/np.trace(a)) #logvariance features 
        return f
    
    def train(self, trainCOV, trainLabels, n=3):
        
        classSpecificCOV = self.separate_classes(trainCOV, trainLabels)
        means = self.estimateMeans(classSpecificCOV)
        
        V=self.CSP(means, n)
        self.V=V
        self.n=n
        train_features = np.empty((len(trainCOV), 2*n))
        
        for i in range(len(trainCOV)):
            trial = trainCOV[i]
            train_features[i] = self.applyCSP(trial, V)
        
        self.clf.fit(train_features, trainLabels)
    
    def predict(self, testCOV):
        V = self.V
        n = self.n
        
        if V is None or n is None:
            raise Exception('Train the model first')
        
        test_features = np.empty((len(testCOV), 2*n))
        for i in range(len(testCOV)):
            trial = testCOV[i]
            test_features[i] = self.applyCSP(trial, V)
        
        prediction = self.clf.predict(test_features)
        return prediction

In [ ]:
#base classifiers 
c_csp = RCSP('classic', 20, LDA)
airm_csp = RCSP('AIRM', 20, LDA)
lem_csp = RCSP('LEM', 20, LDA)
bw_csp = RCSP('BW', 20, LDA)

def count_accuracy(predicted, true, dec_places=2):
    err_count = 0
    for j in range(len(true)):
        if predicted[j]!=true[j]:
            err_count+=1
    acc = (1-err_count/len(true))*100
    acc = round(acc, dec_places)
    return acc

SubjectsCOV1, SubjectsY1 = pickle.load(open('datasets/54COVSess01.pickle','rb'))
SubjectsCOV2, SubjectsY2 = pickle.load(open('datasets/54COVSess02.pickle','rb'))

# Majority Vote

In [ ]:
def majority_vote(SubjectsCOV, SubjectsY, n):
    df = pd.DataFrame(index=list(range(1, 55))+['Average', 'p-value'], columns=['Classic CSP', 'AIRM CSP', 'LEM CSP', 'BW CSP', 'Majority Vote'])
    
    c_results = [] #very sloppy coding, did not prioritize clean code
    a_results = []
    l_results = []
    bw_results = []
    m_results = []
    
    for i in range(len(SubjectsCOV)):
        SC = deepcopy(SubjectsCOV)
        SY = deepcopy(SubjectsY)
            
        testCOV = SC.pop(i)
        testY = SY.pop(i)
            
        trainCOV = None
        trainY = None

        for j in range(len(SubjectsCOV)-1):
            if trainCOV is None:
                trainCOV = SC[j]
                trainY = SY[j]
            else:
                trainCOV = np.concatenate((trainCOV, SC[j]))
                trainY = np.concatenate((trainY, SY[j]))
        
        c_csp.train(trainCOV, trainY, n=n)
        c_res = c_csp.predict(testCOV)

        airm_csp.train(trainCOV, trainY, n=n)
        a_res = airm_csp.predict(testCOV)

        lem_csp.train(trainCOV, trainY, n=n)
        l_res = lem_csp.predict(testCOV)

        bw_csp.train(trainCOV, trainY, n=n)
        bw_res = bw_csp.predict(testCOV)

        res = np.column_stack((a_res, l_res, bw_csp))
        N = len(res)

        majority = np.empty(N, dtype='uint8')

        for j in range(N):
            majority[j] = np.argmax(np.bincount(res[j]))
    
        c_results.append(count_accuracy(c_res, testY))
        a_results.append(count_accuracy(a_res, testY))
        l_results.append(count_accuracy(l_res, testY))
        bw_results.append(count_accuracy(bw_res, testY))
        m_results.append(count_accuracy(majority, testY))
    
    c_results.append(sum(c_results)/len(c_results)) #чуть чуть клоунский код, но работает
    a_results.append(sum(a_results)/len(a_results))
    l_results.append(sum(l_results)/len(l_results))
    bw_results.append(sum(bw_results)/len(bw_results))
    m_results.append(sum(m_results)/len(m_results))

    a_results.append(wilcoxon(a_results[:-1], c_results[:-1])[1])
    l_results.append(wilcoxon(l_results[:-1], c_results[:-1])[1])
    bw_results.append(wilcoxon(bw_results[:-1], c_results[:-1])[1])
    m_results.append(wilcoxon(m_results[:-1], c_results[:-1])[1])


    df['Classic CSP'] = c_results + [None]
    df['AIRM CSP'] = a_results
    df['LEM CSP'] = l_results
    df['BW CSP'] = bw_results
    df['Majority Vote'] = m_results
    
    return df

## Session 1

In [ ]:
dfmv1_2 = majority_vote(SubjectsCOV1, SubjectsY1, 2)
dfmv1_2.head()

In [ ]:
dfmv1_3 = majority_vote(SubjectsCOV1, SubjectsY1, 3)
dfmv1_3.head()

In [ ]:
dfmv1_4 = majority_vote(SubjectsCOV1, SubjectsY1, 4)
dfmv1_4.head()

In [ ]:
dfmv1_5 = majority_vote(SubjectsCOV1, SubjectsY1, 5)
dfmv1_5.head()

In [ ]:
ResultsSess01 = [dfmv1_2, dfmv1_3, dfmv1_4, dfmv1_5]

filename = 'datasets/MVResultsSess01.pickle'
outfile = open(filename,'wb')
pickle.dump(ResultsSess01, outfile)
outfile.close()


## Session 2

In [ ]:
dfmv2_2 = majority_vote(SubjectsCOV2, SubjectsY2, 2)
dfmv2_2.head()

In [ ]:
dfmv2_3 = majority_vote(SubjectsCOV2, SubjectsY2, 2)
dfmv2_3.head()

In [ ]:
dfmv2_4 = majority_vote(SubjectsCOV2, SubjectsY2, 4)
dfmv2_4.head()

In [ ]:
dfmv2_5 = majority_vote(SubjectsCOV2, SubjectsY2, 5)
dfmv2_5.head()

In [ ]:
ResultsSess02 = [dfmv2_2, dfmv2_3, dfmv2_4, dfmv2_5]

filename = 'datasets/MVResultsSess02.pickle'
outfile = open(filename,'wb')
pickle.dump(ResultsSess02, outfile)
outfile.close()

# Bootstrap Mean Estimation

In [ ]:
class RCSP_bootstrap(RCSP):
    def train(self, trainCOV, trainLabels, n=3, btsp_size=0.6, btsp_n=10):
        btsp_means = np.zeros((2, btsp_n, self.nchannels, self.nchannels))
        trials = len(trainCOV)
        idxs = list(range(trials))
        
        for i in range(btsp_n):
            idx = np.random.choice(idxs, int(btsp_size*trials))
            subsetCOV = trainCOV[idx]
            subsetY = trainLabels[idx]
            
            classSpecificCOV = self.separate_classes(subsetCOV, subsetY)
            means = self.estimateMeans(classSpecificCOV)
            btsp_means[0, i, :, :] = means[0]
            btsp_means[1, i, :, :] = means[1]
        
        btsp_means = self.estimateMeans(btsp_means)
        
        V = self.CSP(btsp_means, n)
        self.V=V
        self.n=n
        
        train_features = np.empty((len(trainCOV), 2*n))
        
        for i in range(len(trainCOV)):
            trial = trainCOV[i]
            train_features[i] = self.applyCSP(trial, V)
        
        self.clf.fit(train_features, trainLabels)    

In [ ]:
c_rcsp_b = RCSP_bootstrap("classic", 20, LDA)
a_rcsp_b = RCSP_bootstrap("AIRM", 20, LDA)
l_rcsp_b = RCSP_bootstrap("LEM", 20, LDA)
bw_rcsp_b = RCSP_bootstrap("BW", 20, LDA)

In [ ]:
def bootstrap_estimation(SubjectsCOV, SubjectsY, n, btsp_size, btsp_n):
    df = pd.DataFrame(index=list(range(1, 55))+['Average', 'p-values'], columns=['Bootstraped Classic CSP', 'Bootstraped AIRM CSP', 'Bootstraped LEM CSP', 'Bootstrapped BW CSP', 'Majority Vote'])
    
    c_results = []
    a_results = []
    l_results = []
    bw_results = []
    m_results = []
    
    for i in range(len(SubjectsCOV)):    
        SC = deepcopy(SubjectsCOV)
        SY = deepcopy(SubjectsY)    
        testCOV = SC.pop(i)
        testY = SY.pop(i)
        
        trainCOV = None
        trainY = None
        
        for j in range(len(SubjectsCOV)-1):
            if trainCOV is None:
                trainCOV = SC[j]
                trainY = SY[j]
            else:
                trainCOV = np.concatenate((trainCOV, SC[j]))
                trainY = np.concatenate((trainY, SY[j]))
        
        
        c_rcsp_b.train(trainCOV, trainY, n, btsp_size, btsp_n)
        c_res = c_rcsp_b.predict(testCOV)
        
        a_rcsp_b.train(trainCOV, trainY, n, btsp_size, btsp_n)
        a_res = a_rcsp_b.predict(testCOV)
        
        l_rcsp_b.train(trainCOV, trainY, n, btsp_size, btsp_n)
        l_res = l_rcsp_b.predict(testCOV)

        bw_rcsp_b.train(trainCOV, trainY, n, btsp_size, btsp_n)
        bw_res = bw_rcsp_b.predict(testCOV)
        
        res = np.column_stack((a_res, l_res, bw_csp))

        N = len(res)

        majority = np.empty(N, dtype='uint8')

        for j in range(N):
            majority[j] = np.argmax(np.bincount(res[j]))


        c_results.append(count_accuracy(c_res, testY))
        a_results.append(count_accuracy(a_res, testY))
        l_results.append(count_accuracy(l_res, testY))
        bw_results.append(count_accuracy(bw_res, testY))
        m_results.append(count_accuracy(majority, testY))
        
    c_results.append(sum(c_results)/len(c_results))
    a_results.append(sum(a_results)/len(a_results))
    l_results.append(sum(l_results)/len(l_results))
    bw_results.append(sum(bw_results)/len(bw_results))
    m_results.append(sum(m_results)/len(m_results))
    

    a_results.append(wilcoxon(a_results[:-1], c_results[:-1])[1])
    l_results.append(wilcoxon(l_results[:-1], c_results[:-1])[1])
    bw_results.append(wilcoxon(bw_results[:-1], c_results[:-1])[1])
    m_results.append(wilcoxon(m_results[:-1], c_results[:-1])[1])
    
    df['Bootstraped Classic CSP'] = c_results + [None]
    df['Bootstraped AIRM CSP'] = a_results
    df['Bootstraped LEM CSP'] = l_results
    df['Bootstraped BW CSP'] = bw_results
    df['Majority Vote'] = m_results
    
    return df
        

In [ ]:
btsp_size = 0.6
btsp_n = 10

## Session 1

In [ ]:
dfbt1_2 = bootstrap_estimation(SubjectsCOV1, SubjectsY1, 2, btsp_size, btsp_n)
dfbt1_2.head()

In [ ]:
dfbt1_3 = bootstrap_estimation(SubjectsCOV1, SubjectsY1, 3, btsp_size, btsp_n)
dfbt1_3.head()

In [ ]:
dfbt1_4 = bootstrap_estimation(SubjectsCOV1, SubjectsY1, 4, btsp_size, btsp_n)
dfbt1_4.head()

In [ ]:
dfbt1_5 = bootstrap_estimation(SubjectsCOV1, SubjectsY1, 5, btsp_size, btsp_n)
dfbt1_5.head()

In [ ]:
ResultsSess01 = [dfbt1_2, dfbt1_3, dfbt1_4, dfbt1_5]

filename = 'datasets/BTResultsSess01.pickle'
outfile = open(filename,'wb')
pickle.dump(ResultsSess01, outfile)
outfile.close()

## Session 2

In [ ]:
dfbt2_2 = bootstrap_estimation(SubjectsCOV2, SubjectsY2, 2, btsp_size, btsp_n)
dfbt2_2.head()

In [ ]:
dfbt2_3 = bootstrap_estimation(SubjectsCOV2, SubjectsY2, 3, btsp_size, btsp_n)
dfbt2_3.head()

In [ ]:
dfbt2_4 = bootstrap_estimation(SubjectsCOV2, SubjectsY2, 4, btsp_size, btsp_n)
dfbt2_4.head()

In [ ]:
dfbt2_5 = bootstrap_estimation(SubjectsCOV2, SubjectsY2, 5, btsp_size, btsp_n)
dfbt2_5.head()

In [ ]:
ResultsSess01 = [dfbt2_2, dfbt2_3, dfbt2_4, dfbt2_5]

filename = 'datasets/BTResultsSess02.pickle'
outfile = open(filename,'wb')
pickle.dump(ResultsSess01, outfile)
outfile.close()